In [41]:
 import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-bright')

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.api import VAR
from statsmodels.tsa.api import VECM
from statsmodels.tsa.vector_ar.vecm import select_coint_rank
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
from statsmodels.tsa.stattools import adfuller
import scipy

/var/folders/76/6_zx5hh17x91wqhg7x9nz_640000gn/T/ipykernel_88359/54534221.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-bright')


In [42]:
import pandas as pd

# Assuming merged_data_5 is the file path
merged_data_5 = 'merged_data_5.xlsx'

# Read data from Excel file into DataFrame
new_data = pd.read_excel(merged_data_5)

In [43]:
new_data

,YearMonth,RON 95,RON 97,DIESEL,selling_gold_price,buying_gold_price,RM/USD_exchange,Overnight_Interest_Rate,CPI,WTI_price
0,2017-03,2.13,2.41,2.11,5803.65,5566.04,4.44,3.00,119.4,49.67
1,2017-04,2.22,2.50,2.15,5944.58,5702.42,4.41,3.00,119.2,51.12
2,2017-05,2.08,2.36,2.01,5707.10,5480.76,4.31,3.00,119.0,48.66
3,2017-06,1.99,2.25,1.91,5730.11,5506.28,4.28,2.99,118.7,45.20
4,2017-07,1.98,2.23,1.96,5624.14,5406.38,4.29,2.98,118.7,46.70
...,...,...,...,...,...,...,...,...,...,...
72,2023-03,2.05,3.35,2.15,9048.91,8689.61,4.47,2.74,129.9,73.37
73,2023-04,2.05,3.35,2.15,9386.67,9014.83,4.42,2.74,130.0,79.44
74,2023-05,2.05,3.35,2.15,9536.10,9162.00,4.52,2.97,130.2,71.74
75,2023-06,2.05,3.37,2.15,9552.95,9179.75,4.63,2.99,130.4,70.37


In [44]:
new_data.set_index('YearMonth', inplace=True)

In [45]:
new_data_copy = new_data[['RON 95', 'RON 97', 'DIESEL','selling_gold_price','buying_gold_price','Overnight_Interest_Rate']].copy()

In [46]:
new_data_copy.shape

(77, 6)

In [47]:
new_data_copy

,RON 95,RON 97,DIESEL,selling_gold_price,buying_gold_price,Overnight_Interest_Rate
YearMonth,,,,,,
2017-03,2.13,2.41,2.11,5803.65,5566.04,3.00
2017-04,2.22,2.50,2.15,5944.58,5702.42,3.00
2017-05,2.08,2.36,2.01,5707.10,5480.76,3.00
2017-06,1.99,2.25,1.91,5730.11,5506.28,2.99
2017-07,1.98,2.23,1.96,5624.14,5406.38,2.98
...,...,...,...,...,...,...
2023-03,2.05,3.35,2.15,9048.91,8689.61,2.74
2023-04,2.05,3.35,2.15,9386.67,9014.83,2.74
2023-05,2.05,3.35,2.15,9536.10,9162.00,2.97


In [48]:
0.6*77

46.199999999999996

In [49]:
df_train = new_data_copy[0:46]
df_test = new_data_copy[46:]

In [50]:
df_train

,RON 95,RON 97,DIESEL,selling_gold_price,buying_gold_price,Overnight_Interest_Rate
YearMonth,,,,,,
2017-03,2.13,2.41,2.11,5803.65,5566.04,3.00
2017-04,2.22,2.50,2.15,5944.58,5702.42,3.00
2017-05,2.08,2.36,2.01,5707.10,5480.76,3.00
2017-06,1.99,2.25,1.91,5730.11,5506.28,2.99
2017-07,1.98,2.23,1.96,5624.14,5406.38,2.98
2017-08,2.13,2.40,2.04,5819.77,5594.23,2.97
2017-09,2.19,2.49,2.10,5871.76,5644.12,2.96
2017-10,2.18,2.48,2.13,5744.71,5379.00,2.97
2017-11,2.31,2.59,2.22,5677.50,5458.32,2.94


In [51]:
df_test

,RON 95,RON 97,DIESEL,selling_gold_price,buying_gold_price,Overnight_Interest_Rate
YearMonth,,,,,,
2021-01,1.87,2.17,2.05,8009.74,7694.84,1.74
2021-02,1.98,2.28,2.14,7784.53,7479.84,1.75
2021-03,2.05,2.43,2.15,7509.09,7215.00,1.72
2021-04,2.05,2.55,2.15,7679.48,7380.62,1.71
2021-05,2.05,2.60,2.15,8092.06,7774.18,1.73
2021-06,2.05,2.66,2.15,8063.35,7748.20,1.72
2021-07,2.05,2.72,2.15,8031.61,7719.44,1.74
2021-08,2.05,2.74,2.15,8001.35,7689.40,1.74
2021-09,2.05,2.74,2.15,7875.05,7568.89,1.74


In [52]:
#We apply the log to make the variance constant
df_train_log = np.log(df_train)

In [53]:
from statsmodels.tsa.stattools import acf, pacf, adfuller

def adf_test(time_series):
    result = adfuller(df_train_log[time_series], autolag="AIC")
    # Extract and print the test statistic, p-value, and critical values
    print('ADF Statistic (%s): %f' % (time_series, result[0]))
    print('p-value (%s): %f' % (time_series, result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')
        
# Perform ADF test for all the time series
for col in df_train_log.columns:
    adf_test(col)
    print('-----------------------------------')

ADF Statistic (RON 95): -1.438272
p-value (RON 95): 0.563797
Critical Values:
   1%: -3.596635636000432
   5%: -2.933297331821618
   10%: -2.6049909750566895
-----------------------------------
ADF Statistic (RON 97): -1.628158
p-value (RON 97): 0.468455
Critical Values:
   1%: -3.5925042342183704
   5%: -2.931549768951162
   10%: -2.60406594375338
-----------------------------------
ADF Statistic (DIESEL): -1.404296
p-value (DIESEL): 0.580215
Critical Values:
   1%: -3.5925042342183704
   5%: -2.931549768951162
   10%: -2.60406594375338
-----------------------------------
ADF Statistic (selling_gold_price): -0.165129
p-value (selling_gold_price): 0.942553
Critical Values:
   1%: -3.5885733964124715
   5%: -2.929885661157025
   10%: -2.6031845661157025
-----------------------------------
ADF Statistic (buying_gold_price): -0.043801
p-value (buying_gold_price): 0.954756
Critical Values:
   1%: -3.5885733964124715
   5%: -2.929885661157025
   10%: -2.6031845661157025
--------------------

In [54]:
 #Let's difference our series once and drop NA values
df_train_log_diff = (df_train_log).diff().dropna()

In [55]:
def adf_test(time_series):
    result = adfuller(df_train_log_diff[time_series], autolag="AIC")
    # Extract and print the test statistic, p-value, and critical values
    print('ADF Statistic (%s): %f' % (time_series, result[0]))
    print('p-value (%s): %f' % (time_series, result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')
        
# Perform ADF test for all the time series
for col in df_train_log_diff.columns:
    adf_test(col)
    print('-----------------------------------')

ADF Statistic (RON 95): -3.992115
p-value (RON 95): 0.001452
Critical Values:
   1%: -3.60098336718852
   5%: -2.9351348158036012
   10%: -2.6059629803688282
-----------------------------------
ADF Statistic (RON 97): -5.371789
p-value (RON 97): 0.000004
Critical Values:
   1%: -3.5925042342183704
   5%: -2.931549768951162
   10%: -2.60406594375338
-----------------------------------
ADF Statistic (DIESEL): -6.685676
p-value (DIESEL): 0.000000
Critical Values:
   1%: -3.5925042342183704
   5%: -2.931549768951162
   10%: -2.60406594375338
-----------------------------------
ADF Statistic (selling_gold_price): -4.629189
p-value (selling_gold_price): 0.000114
Critical Values:
   1%: -3.5885733964124715
   5%: -2.929885661157025
   10%: -2.6031845661157025
-----------------------------------
ADF Statistic (buying_gold_price): -5.053061
p-value (buying_gold_price): 0.000017
Critical Values:
   1%: -3.5885733964124715
   5%: -2.929885661157025
   10%: -2.6031845661157025
--------------------

In [56]:
from statsmodels.tsa.api import VAR

#Initiate VAR model
var_model = VAR(endog=df_train_log_diff)
res_var = var_model.select_order(maxlags=5, trend='ct')
res_var.summary()

/Users/sitinurularisya/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


,AIC,BIC,FPE,HQIC
0,-43.86,-43.36*,8.937e-20,-43.68
1,-43.53,-41.50,1.288e-19,-42.80
2,-44.07,-40.52,8.760e-20,-42.78
3,-45.34,-40.27,3.686e-20,-43.51
4,-47.75,-41.17,8.211e-21,-45.37
5,-51.43*,-43.32,1.661e-21*,-48.50*


In [57]:
var_model_fit = var_model.fit(maxlags=5, ic='aic')
var_model_fit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 12, May, 2024
Time:                     18:48:55
--------------------------------------------------------------------
No. of Equations:         6.00000    BIC:                   -42.5717
Nobs:                     40.0000    HQIC:                  -47.5855
Log likelihood:           853.955    FPE:                2.77881e-21
AIC:                     -50.4250    Det(Omega_mle):     8.88524e-23
--------------------------------------------------------------------
Results for equation RON 95
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                             -0.006637         0.009082           -0.731           0.465
L1.RON 95                         -1.997176         0.632618           -3.157           0.002
L1.RON 

In [58]:
from statsmodels.stats.stattools import durbin_watson

dw_check_var = durbin_watson(var_model_fit.resid)

for col, val in zip(df_train_log_diff.columns, dw_check_var):
    print("Durbin-Watson statistic for:")
    print(col, '-->', round(val, 2))
    print("----------------------------")

Durbin-Watson statistic for:
RON 95 --> 1.87
----------------------------
Durbin-Watson statistic for:
RON 97 --> 1.78
----------------------------
Durbin-Watson statistic for:
DIESEL --> 1.88
----------------------------
Durbin-Watson statistic for:
selling_gold_price --> 2.2
----------------------------
Durbin-Watson statistic for:
buying_gold_price --> 2.1
----------------------------
Durbin-Watson statistic for:
Overnight_Interest_Rate --> 2.26
----------------------------


In [59]:
############################# prediction part ###############################

In [60]:
#Get the lag order
lag_order = var_model_fit.k_ar
print(lag_order)

5


In [61]:
#Input data for forecasting
pred_input = df_train_log_diff.values[-lag_order:]
pred_input

array([[-0.00593474, -0.00503779, -0.02739897,  0.0565679 ,  0.0564739 ,
        -0.02833051],
       [-0.00597017, -0.0050633 , -0.04546237, -0.03462556, -0.034854  ,
        -0.0057637 ],
       [ 0.        ,  0.        ,  0.        , -0.01272162, -0.01283249,
         0.01149438],
       [-0.01812738, -0.01534557,  0.02865526, -0.02550688, -0.02543392,
        -0.00573067],
       [ 0.06491955,  0.05515182,  0.09170843, -0.02201442, -0.02192661,
         0.        ]])

In [62]:
#Forecasting
var_pred = var_model_fit.forecast(y=pred_input, steps=len(df_test))

df_var_pred = pd.DataFrame(var_pred, index = df_test_log.index, columns = df_train_log_diff.columns + '_1m')

df_var_pred.tail()

,RON 95_1m,RON 97_1m,DIESEL_1m,selling_gold_price_1m,buying_gold_price_1m,Overnight_Interest_Rate_1m
YearMonth,,,,,,
2023-03,-0.042920,-0.039049,-0.031111,0.018304,0.017026,0.001249
2023-04,-0.021247,-0.009383,-0.014360,0.010934,0.010285,-0.015608
2023-05,0.009228,0.007497,0.015372,0.008211,0.009305,-0.028753
2023-06,0.013409,0.019274,0.014792,0.002118,0.002912,-0.019006
2023-07,-0.001935,0.001631,-0.004539,0.013722,0.013167,-0.013427


In [63]:
df_var_pred.shape

(31, 6)

In [64]:
# inverting transformation: single difference
def invert_transformation(df_train, df_forecast):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = np.log(df_train[col].iloc[-1]) + df_fc[str(col)+'_1m'].cumsum()
    return df_fc

In [65]:
df_var_pred_inv = invert_transformation(df_train, df_var_pred)
df_var_pred_inv = df_var_pred_inv.filter(regex='_forecast$',axis=1)
df_var_pred_inv = np.exp(df_var_pred_inv) 

round(df_var_pred_inv.head(), 2)

,RON 95_forecast,RON 97_forecast,DIESEL_forecast,selling_gold_price_forecast,buying_gold_price_forecast,Overnight_Interest_Rate_forecast
YearMonth,,,,,,
2021-01,2.02,2.44,2.12,7498.15,7249.74,1.89
2021-02,2.13,2.68,2.18,7250.96,6951.12,2.00
2021-03,2.12,2.74,2.20,7204.28,6936.45,2.07
2021-04,2.00,2.40,2.10,7165.40,6922.64,2.08
2021-05,1.81,2.20,1.99,7175.46,6912.49,2.21


In [66]:
import numpy as np

# Assuming df_test is your actual data and df_var_pred_inv are the predictions from your VAR model

# Calculate MAPE
var_mape = metrics.mean_absolute_percentage_error(df_test, df_var_pred_inv)

# Calculate MAE
var_mae = metrics.mean_absolute_error(df_test, df_var_pred_inv)

# Calculate RMSE
var_rmse = np.sqrt(metrics.mean_squared_error(df_test, df_var_pred_inv))

# Calculate the adjusted accuracy-like metric based on MAPE
var_model_accuracy = 1 - var_mape

# Print all the calculated metrics
print("The accuracy of the VAR out-of-sample prediction is:")
print("----------------------------------------------------")
print(f"Accuracy: {round(var_model_accuracy, 3)}")
print(f"MAE: {round(var_mae, 3)}")
print(f"RMSE: {round(var_rmse, 3)}")
print(f"MAPE: {round(var_mape, 3)}")

The accuracy of the VAR out-of-sample prediction is:
----------------------------------------------------
Accuracy: 0.837
MAE: 121.09
RMSE: 249.094
MAPE: 0.163
